# Getting Started: Voice swap application
This notebook shows how to use NVIDIA NeMo (https://github.com/NVIDIA/NeMo) to construct a toy demo which will swap a voice in the audio fragment with a computer generated one.

At its core the demo does: 

* Automatic speech recognition of what is said in the file. E.g. converting audio to text
* Adding punctuation and capitalization to the text
* Generating spectrogram from resulting text
* Generating waveform audio from the spectrogram.

## Installation
NeMo can be installed via simple pip command.

In [1]:
BRANCH = 'r1.6.0'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]


  Cloning https://github.com/NVIDIA/NeMo.git (to revision r1.6.0) to /tmp/pip-install-mjvtjl69/nemo-toolkit_3ae16707743c4cd2b95c32ee7c690278
  Running command git clone -q https://github.com/NVIDIA/NeMo.git /tmp/pip-install-mjvtjl69/nemo-toolkit_3ae16707743c4cd2b95c32ee7c690278
  Running command git checkout -b r1.6.0 --track origin/r1.6.0
  Switched to a new branch 'r1.6.0'
  Branch 'r1.6.0' set up to track remote branch 'r1.6.0' from 'origin'.
     |████████████████████████████████| 15.7 MB 8.6 MB/s 
     |████████████████████████████████| 12.7 MB 62.4 MB/s 
     |████████████████████████████████| 109 kB 55.5 MB/s 
     |████████████████████████████████| 1.2 MB 57.0 MB/s 
     |████████████████████████████████| 92 kB 1.2 MB/s 
     |████████████████████████████████| 235 kB 58.3 MB/s 
     |████████████████████████████████| 97 kB 8.1 MB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 
  Using cached pytest_runner-5.3.1-py3-none-any.whl (7.1 kB)
     |████████████████████████

In [2]:
# Ignore pre-production warnings
import warnings
warnings.filterwarnings('ignore')
import nemo
# Import Speech Recognition collection
import nemo.collections.asr as nemo_asr
# Import Natural Language Processing colleciton
import nemo.collections.nlp as nemo_nlp
# Import Speech Synthesis collection
import nemo.collections.tts as nemo_tts
# We'll use this to listen to audio
import IPython

[NeMo W 2022-01-31 16:08:27 optimizers:52] Apex was not found. Using the lamb or fused_adam optimizer will error out.
################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################

[NeMo W 2022-01-31 16:08:28 experimental:28] Module <function get_argmin_mat at 0x7fbfbf002680> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022-01-31 16:08:28 experimental:28] Module <function getMultiScaleCosAffinityMatrix at 0x7fbfbf1dd830> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022-01-31 16:08:28 experimental:28] Module <function parse_scale_configs at 0x

In [3]:
# Download audio sample which we'll try
# This is a sample from LibriSpeech Dev Clean dataset - the model hasn't seen it before
Audio_sample = '2086-149220-0033.wav'
!wget https://dldata-public.s3.us-east-2.amazonaws.com/2086-149220-0033.wav
# Listen to it
IPython.display.Audio(Audio_sample)

--2022-01-31 16:15:43--  https://dldata-public.s3.us-east-2.amazonaws.com/2086-149220-0033.wav
Resolving dldata-public.s3.us-east-2.amazonaws.com (dldata-public.s3.us-east-2.amazonaws.com)... 52.219.103.10
Connecting to dldata-public.s3.us-east-2.amazonaws.com (dldata-public.s3.us-east-2.amazonaws.com)|52.219.103.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 237964 (232K) [audio/wav]
Saving to: ‘2086-149220-0033.wav’

2086-149220-0033.wa 100%[===================>] 232.39K  1.19MB/s    in 0.2s    

2022-01-31 16:15:44 (1.19 MB/s) - ‘2086-149220-0033.wav’ saved [237964/237964]



## Instantiate pre-trained NeMo models which we'll use
``from_pretrained(...)`` API downloads and initialized model directly from the cloud.

We will load audio_sample and convert it to text with QuartzNet ASR model (an action called transcribe).
To convert text back to audio, we actually need to generate spectrogram with FastPitch first and then convert it to actual audio signal using the HiFiGAN vocoder.

In [4]:
# Speech Recognition model - QuartzNet
quartznet = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="stt_en_quartznet15x5").cuda()
# Punctuation and capitalization model
punctuation = nemo_nlp.models.PunctuationCapitalizationModel.from_pretrained(model_name='punctuation_en_distilbert').cuda()
# Spectrogram generator which takes text as an input and produces spectrogram
spectrogram_generator = nemo_tts.models.FastPitchModel.from_pretrained(model_name="tts_en_fastpitch").cuda()
# Vocoder model which takes spectrogram and produces actual audio
vocoder = nemo_tts.models.HifiGanModel.from_pretrained(model_name="tts_hifigan").cuda()

[NeMo I 2022-01-31 16:16:03 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/stt_en_quartznet15x5/versions/1.0.0rc1/files/stt_en_quartznet15x5.nemo to /root/.cache/torch/NeMo/NeMo_1.6.0/stt_en_quartznet15x5/16661021d16e679bdfd97a2a03944c49/stt_en_quartznet15x5.nemo
[NeMo I 2022-01-31 16:16:05 common:728] Instantiating model from pre-trained checkpoint


[NeMo W 2022-01-31 16:16:06 modelPT:136] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data2/voices/train_1k.json
    sample_rate: 16000
    labels:
    - ' '
    - a
    - b
    - c
    - d
    - e
    - f
    - g
    - h
    - i
    - j
    - k
    - l
    - m
    - 'n'
    - o
    - p
    - q
    - r
    - s
    - t
    - u
    - v
    - w
    - x
    - 'y'
    - z
    - ''''
    batch_size: 32
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: /asr_set_1.2/train/train_{0..1023}.tar
    num_workers: 20
    
[NeMo W 2022-01-31 16:16:06 modelPT:143] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
   

[NeMo I 2022-01-31 16:16:06 features:264] PADDING: 16
[NeMo I 2022-01-31 16:16:06 features:281] STFT using torch
[NeMo I 2022-01-31 16:16:18 save_restore_connector:154] Model EncDecCTCModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.6.0/stt_en_quartznet15x5/16661021d16e679bdfd97a2a03944c49/stt_en_quartznet15x5.nemo.
[NeMo I 2022-01-31 16:16:18 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/punctuation_en_distilbert/versions/1.0.0rc1/files/punctuation_en_distilbert.nemo to /root/.cache/torch/NeMo/NeMo_1.6.0/punctuation_en_distilbert/613c4ee780c6fc158f49d3566cbd6636/punctuation_en_distilbert.nemo
[NeMo I 2022-01-31 16:16:23 common:728] Instantiating model from pre-trained checkpoint
[NeMo I 2022-01-31 16:16:26 tokenizer_utils:126] Getting HuggingFace AutoTokenizer with pretrained_model_name: distilbert-base-uncased, vocab_file: /tmp/tmpdmvrrt8v/tokenizer.vocab_file, special_tokens_dict: {}, and use_fast: False


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

...

[NeMo I 2022-01-31 16:17:01 features:264] PADDING: 1
[NeMo I 2022-01-31 16:17:01 features:281] STFT using torch
[NeMo I 2022-01-31 16:17:02 save_restore_connector:154] Model FastPitchModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.6.0/tts_en_fastpitch_align/b50e16c5d695b00855ae53d6ba4e4f7f/tts_en_fastpitch_align.nemo.
[NeMo I 2022-01-31 16:17:02 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/tts_hifigan/versions/1.0.0rc1/files/tts_hifigan.nemo to /root/.cache/torch/NeMo/NeMo_1.6.0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo
[NeMo I 2022-01-31 16:17:08 common:728] Instantiating model from pre-trained checkpoint


[NeMo W 2022-01-31 16:17:12 modelPT:136] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/train_finetune.txt
      min_duration: 0.75
      n_segments: 8192
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 64
      num_workers: 4
    
[NeMo W 2022-01-31 16:17:12 modelPT:143] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/val_finetune.txt
      min_duration: 3
      n_segments: 66150


[NeMo I 2022-01-31 16:17:12 features:264] PADDING: 0
[NeMo I 2022-01-31 16:17:12 features:281] STFT using torch


[NeMo W 2022-01-31 16:17:12 features:242] Using torch_stft is deprecated and will be removed in 1.1.0. Please set stft_conv and stft_exact_pad to False for FilterbankFeatures and AudioToMelSpectrogramPreprocessor. Please set exact_pad to True as needed.


[NeMo I 2022-01-31 16:17:12 features:264] PADDING: 0
[NeMo I 2022-01-31 16:17:12 features:281] STFT using torch
[NeMo I 2022-01-31 16:17:13 save_restore_connector:154] Model HifiGanModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.6.0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.


## Using the models

In [5]:
# Convert our audio sample to text
files = [Audio_sample]
raw_text = ''
text = ''
for fname, transcription in zip(files, quartznet.transcribe(paths2audio_files=files)):
  raw_text = transcription

# Add capitalization and punctuation
res = punctuation.add_punctuation_capitalization(queries=[raw_text])
text = res[0]
print(f'\nRaw recognized text: {raw_text}. \nText with capitalization and punctuation: {text}')

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

[NeMo I 2022-01-31 16:17:14 punctuation_capitalization_model:1068] Using batch size 1 for inference
[NeMo I 2022-01-31 16:17:14 punctuation_capitalization_infer_dataset:91] Max length: 28
[NeMo I 2022-01-31 16:17:14 data_preprocessing:358] Some stats of the lengths of the sequences:
[NeMo I 2022-01-31 16:17:14 data_preprocessing:364] Min: 26 |                  Max: 26 |                  Mean: 26.0 |                  Median: 26.0
[NeMo I 2022-01-31 16:17:14 data_preprocessing:366] 75 percentile: 26.00
[NeMo I 2022-01-31 16:17:14 data_preprocessing:367] 99 percentile: 26.00


100%|██████████| 1/1 [00:00<00:00, 25.83batch/s]


Raw recognized text: well i don't wish to see it any more observed phoebe turning away her eyes it is certainly very likt the old portrait. 
Text with capitalization and punctuation: Well, I don't wish to see it any more, observed Phoebe, turning away her eyes. It is certainly very likt the old portrait.


In [6]:
# A helper function which combines TTS models to go directly from 
# text to audio
def text_to_audio(text):
  parsed = spectrogram_generator.parse(text)
  spectrogram = spectrogram_generator.generate_spectrogram(tokens=parsed)
  audio = vocoder.convert_spectrogram_to_audio(spec=spectrogram)
  return audio.to('cpu').detach().numpy()

## Results

In [7]:
# This is our original audio sample
IPython.display.Audio(Audio_sample)

In [8]:
# This is what was recognized by the ASR model
print(raw_text)

well i don't wish to see it any more observed phoebe turning away her eyes it is certainly very likt the old portrait


In [12]:
# This is how punctuation model changed it
print(text)

Well, I don't wish to see it any more, observed Phoebe, turning away her eyes. It is certainly very likt the old portrait.


Compare how the synthesized audio sounds when using text with and without punctuation.

In [14]:
# Without punctuation
IPython.display.Audio(text_to_audio(raw_text), rate=22050)

In [11]:
# Final result - with punctuation
IPython.display.Audio(text_to_audio(text), rate=22050)

## Next steps
A demo like this is great for prototyping and experimentation. However, for real production deployment, you would want to use a service like [NVIDIA Riva](https://developer.nvidia.com/riva).

**NeMo is built for training.** You can fine-tune, or train from scratch on your data all models used in this example. We recommend you checkout the following, more in-depth, tutorials next:

* [NeMo fundamentals](https://colab.research.google.com/github/NVIDIA/NeMo/blob/stable/tutorials/00_NeMo_Primer.ipynb)
* [NeMo models](https://colab.research.google.com/github/NVIDIA/NeMo/blob/stable/tutorials/01_NeMo_Models.ipynb)
* [Speech Recognition](https://colab.research.google.com/github/NVIDIA/NeMo/blob/stable/tutorials/asr/ASR_with_NeMo.ipynb)
* [Punctuation and Capitalization](https://colab.research.google.com/github/NVIDIA/NeMo/blob/stable/tutorials/nlp/Punctuation_and_Capitalization.ipynb)
* [Speech Synthesis](https://colab.research.google.com/github/NVIDIA/NeMo/blob/stable/tutorials/tts/Inference_ModelSelect.ipynb)


You can find scripts for training and fine-tuning ASR, NLP and TTS models [here](https://github.com/NVIDIA/NeMo/tree/main/examples). 

That's it folks! Head over to NeMo GitHub for more examples: https://github.com/NVIDIA/NeMo